# kaggle score: 0.5473 R^2 - Gradient Boost Model¶

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt       
%matplotlib inline 
import seaborn as sns
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_selection import VarianceThreshold, mutual_info_regression, RFE
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split, RandomizedSearchCV
import json
from scipy.stats import randint as sp_randint

In [ ]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/merc/train.csv")
train

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8405,107.39,ak,s,as,c,d,aa,d,q,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4205,8406,108.77,j,o,t,d,d,aa,h,h,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4206,8412,109.22,ak,v,r,a,d,aa,g,e,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4207,8415,87.48,al,r,e,f,d,aa,l,u,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/merc/test.csv")
test

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8410,aj,h,as,f,d,aa,j,e,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4205,8411,t,aa,ai,d,d,aa,j,y,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4206,8413,y,v,as,f,d,aa,d,w,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4207,8414,ak,v,as,a,d,aa,c,q,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
y = train["y"]
train = train.drop(["ID","y"], axis=1) 

In [ ]:
with open("/content/drive/My Drive/Colab Notebooks/merc/rf_features.txt", "r") as fp:
  rf_binfeats = json.load(fp)

with open("/content/drive/My Drive/Colab Notebooks/merc/gb_features.txt", "r") as fp:
  gb_binfeats = json.load(fp)

with open("/content/drive/My Drive/Colab Notebooks/merc/rf_catfeatures.txt", "r") as fp:
  rf_catfeats = json.load(fp)

with open("/content/drive/My Drive/Colab Notebooks/merc/gb_catfeatures.txt", "r") as fp:
  gb_catfeats = json.load(fp)

**Combining categorical (dummy encoded) and binary features (for respective models) that we have selected in the previous part:**

In [ ]:
rf_cols = rf_binfeats + rf_catfeats
gb_cols = gb_binfeats + gb_catfeats

print("total columns selected for randomforest: {}".format(len(rf_cols)))
print("total columns selected for gradientboost: {}".format(len(gb_cols)))

total columns selected for randomforest: 127
total columns selected for gradientboost: 56


**Making train and test sets for randomforest and gradientboosting model:**

In [ ]:
catcols = train.iloc[:,:8].columns.to_list()
catcols.remove("X4")
catcols

['X0', 'X1', 'X2', 'X3', 'X5', 'X6', 'X8']

In [ ]:
train = pd.get_dummies(train,columns=catcols)
test = pd.get_dummies(test,columns=catcols)

In [ ]:
intersection_cols = set(train.columns).intersection(set(test.columns))
len(intersection_cols)

550

In [ ]:
xtr_rf = train[rf_cols]
xtr_gb = train[gb_cols]

xte_rf = test[rf_cols]
xte_gb = test[gb_cols]

print("train sets for randomforest and gradientboosting:")
print(xtr_rf.shape)
print(xtr_gb.shape)

print("test sets for randomforest and gradientboosting:")
print(xte_rf.shape)
print(xte_gb.shape)

train sets for randomforest and gradientboosting:
(4209, 127)
(4209, 56)
test sets for randomforest and gradientboosting:
(4209, 127)
(4209, 56)


**Defining some functions which we will be using:**

In [ ]:
def training_and_tuning(model,xtr,ytr,parameters,iterations):
  print("started random search cv:\n")
  randomCV = RandomizedSearchCV(model, param_distributions=parameters, n_iter=iterations, cv=5, random_state=7)
  randomCV.fit(xtr, ytr)
  best = randomCV.best_params_
  best_sc = randomCV.best_score_
  print("best score:")
  print(best_sc)
  print("best parameters obtained:")
  print(best)
  return best

In [ ]:
def tuned_model(cl,xtr,ytr):
  scores = cross_val_score(cl, xtr, ytr, cv=5)
  print("5 cv score: {}".format(scores.mean()))

In [ ]:
def kaggle_testset(final_model,trainset,trainlabels,testset,filename):
  final_model.fit(trainset,trainlabels)
  final_predictions = final_model.predict(testset)

  submit = pd.DataFrame(columns=["ID","y"])
  submit["y"] = final_predictions
  submit["ID"] = test["ID"]
  submit.to_csv("/content/drive/My Drive/Colab Notebooks/merc/"+filename,index=False)

# random forest:

In [ ]:
%%time
model = RandomForestRegressor(random_state=7)

parameters = {"max_features": list(np.arange(1,30))+[None,"sqrt"],
              "max_depth":list(np.arange(3,50))+[None],
              "n_estimators": np.arange(10,150,5)}

best = training_and_tuning(model,xtr_rf,y,parameters,100)

started random search cv:

best score:
0.5600429556262917
best parameters obtained:
{'n_estimators': 145, 'max_features': None, 'max_depth': 9}
CPU times: user 5min 3s, sys: 753 ms, total: 5min 4s
Wall time: 5min 5s


In [ ]:
cl = RandomForestRegressor(n_estimators=145, max_features=None, max_depth=9, random_state=7)
tuned_model(cl, xtr_rf, y)

5 cv score: 0.5600429556262917


In [ ]:
# kaggle_testset(cl, xtr_rf, y, xte_rf, "rf_submission.csv")

**kaggle score: 0.5292 (private leaderboard)**
# GradientBoosting:

In [ ]:
%%time
model = GradientBoostingRegressor(random_state=7)

parameters = {"max_features": list(np.arange(1,30))+[None,"sqrt"],
              "max_depth":list(np.arange(3,50))+[None],
              "n_estimators": np.arange(10,150,5)}

best = training_and_tuning(model,xtr_gb,y,parameters,100)

started random search cv:

best score:
0.5709360915535797
best parameters obtained:
{'n_estimators': 50, 'max_features': 28, 'max_depth': 3}
CPU times: user 3min 23s, sys: 126 ms, total: 3min 24s
Wall time: 3min 24s


In [ ]:
cl = GradientBoostingRegressor(n_estimators=50, max_features=28, max_depth=3, random_state=7)
tuned_model(cl, xtr_gb, y)

5 cv score: 0.5709360915535797


In [ ]:
# kaggle_testset(cl, xtr_gb, y, xte_gb, "gb_submission.csv")

**kaggle score: 0.5473 (private leaderboard)**

# Kaggle winning score on private leaderboard: 0.5555
### Now in 3rd part we will not dummy encode the categorical features. Instead we will do normalized frequency encoding